In [1]:
from dotenv import load_dotenv 

load_dotenv()

True

In [2]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: list
    answer: str
    
graph_builder = StateGraph(AgentState)

In [3]:
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

def web_search(state: AgentState) -> AgentState:
    """
    주어진 state를 기반으로 웹 검색을 수행합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 웹 검색 결과가 추가된 state를 반환합니다.
    """
    query = state['query']
    results = tavily_search_tool.invoke(query)

    return {'context': results}

In [4]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LangChain 허브에서 프롬프트를 가져옵니다
generate_prompt = hub.pull("rlm/rag-prompt")
# OpenAI의 GPT-4o 모델을 사용합니다
generate_llm = ChatOpenAI(model="gpt-4o")

def web_generate(state: AgentState) -> AgentState:
    """
    주어진 문맥과 질문을 기반으로 답변을 생성합니다.

    Args:
        state (AgentState): 문맥과 질문을 포함한 에이 트의 현재 state.

    Returns:
        AgentState: 생성된 답변을 포함한 state를 반환합니다.
    """
    # state에서 문맥과 질문을 추출합니다
    context = state['context']
    query = state['query']
    
    # 프롬프트와 모델, 출력 파서를 연결하여 체인을 생성합니다
    rag_chain = generate_prompt | generate_llm | StrOutputParser()
    
    # 체인을 사용하여 답변을 생성합니다
    response = rag_chain.invoke({'question': query, 'context': context})
    
    # 생성된 답변을 'answer'로 반환합니다
    return {'answer': response}

/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# OpenAI의 GPT-4o-mini 모델을 사용합니다
basic_llm = ChatOpenAI(model="gpt-4o-mini")

def basic_generate(state: AgentState) -> AgentState:
    """
    사용자의 질문에 기반하여 기본 답변을 생성합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 생성된 답변을 포함한 state를 반환합니다.
    """
    # state에서 질문을 추출합니다
    query = state['query']
    
    # 기본 LLM 체인을 생성합니다
    basic_llm_chain = basic_llm | StrOutputParser()
    
    # 체인을 사용하여 답변을 생성합니다
    llm_response = basic_llm_chain.invoke(query)
    
    # 생성된 답변을 'answer'로 반환합니다
    return {'answer': llm_response}

In [6]:
from langchain_core.prompts import ChatPromptTemplate 
from pydantic import BaseModel, Field 
from typing import Literal 

class Route(BaseModel):
    target: Literal['vector_store', 'llm', 'web_search'] = Field(
        description="the target for the query to answer"
    )

router_system_prompt  = '''
You are an expert at routing a user's question to 'vector_store', 'llm', or 'web_search'.
'vector_store' contains information about income tax up to December 2024.
if you think the question is simple enough use 'llm'
if you think you need to search the web to answer the question use 'web_search'
'''

router_prompt = ChatPromptTemplate.from_messages([
    ("system", router_system_prompt),
    ("user", '{query}')
])

router_llm = ChatOpenAI(model='gpt-4o-mini')
structured_router_llm = router_llm.with_structured_output(Route)

def router(state: AgentState) -> Literal["vector_store", "llm", "web_search"]:
    query = state['query']
    router_chain = router_prompt | structured_router_llm

    route = router_chain.invoke({"query": query})

    return route.target

In [7]:
from income_tax_graph import graph as income_tax_subgraph

graph_builder.add_node('income_tax_agent', income_tax_subgraph)
graph_builder.add_node('web_search', web_search)
graph_builder.add_node('web_generate', web_generate)
graph_builder.add_node('basic_generate', basic_generate)

/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from langgraph.graph import START, END 

graph_builder.add_conditional_edges(
    START,
    router,
    {
        'vector_store': 'income_tax_agent',
        'llm': 'basic_generate',
        'web_search': 'web_search'
    }
)

graph_builder.add_edge('web_search', 'web_generate')
graph_builder.add_edge('web_generate', END)
graph_builder.add_edge('basic_generate', END)
graph_builder.add_edge('income_tax_agent', END)


In [9]:
graph = graph_builder.compile()

In [11]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))
graph.get_graph().print_ascii()


                              +-----------+                                  
                              | __start__ |..                                
                         .....+-----------+  .....                           
                     ....            .            .....                      
                .....               .                  ....                  
             ...                    .                      .....             
 +------------+                     .                           ...          
 | web_search |                     .                             .          
 +------------+                     .                             .          
        *                           .                             .          
        *                           .                             .          
        *                           .                             .          
+--------------+           +----------------+           +-------

In [12]:
initial_state = {'query': '대한민국의 수도는 어디인가요?'}
graph.invoke(initial_state)

{'query': '대한민국의 수도는 어디인가요?', 'answer': '대한민국의 수도는 서울입니다.'}

In [13]:
initial_state = {'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?'}
graph.invoke(initial_state)

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'context': [Document(metadata={'source': './documents/income_tax.txt'}, page_content='| 1,400만원 초과     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)                    |\n| 5,000만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)                  |\n| 8,800만원 이하     | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)                |\n| 1억5천만원 이하    | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)                |\n| 3억원 이하        | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)                    |\n| 5억원 이하        | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)                 |\n| 10억원 이하       | 3억3,406만원 + (10억원을 초과하는 금액의 45퍼센트)                |\n법제처 35 국가법령정보센터\n소득세법\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 "퇴직소득 산출세액"이라 한다)으로 한다. <개정 2013. 1. 1, 2014. 12. 23.>\n   1. 해당 과세기간의 퇴직소득과세표준에 제11항의 세율을 적용하여 계산한 금액\n   2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n   3. 삭제 <2014. 12. 23.>\n   [전문개정 2009. 12. 31.]\n제2관 세액공제 <개정 2009. 12. 31.>\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은

In [14]:
initial_state = {'query': '역삼 맛집을 추천해주세요'}
graph.invoke(initial_state)

{'query': '역삼 맛집을 추천해주세요',
 'context': [{'url': 'https://foodtriplove.com/entry/역삼역-맛집-BEST-5-모두의-맛집',
   'content': '음식공감\n\n음식공감\n\n음식공감\n\n태그\n\n최근글\n\n댓글\n\n공지사항\n\n아카이브\n\n2022. 10. 5. 17:00ㆍ맛집리스트\n\n본 블로그의 맛집 선정 기준은 네이버에 나온 방문자, 블로그 리뷰 수와 리뷰 글들을 보고 선정했습니다. 한번에 보실 수 있도록 기획해봤습니다. 리뷰 글들 꼭 참고하시는 걸 추천드립니다. 그럼 역삼역 맛집을 소개해드리겠습니다.\n\n1. 역삼역 맛집 " 스터번 "\n\n-\xa0뉴욕 스테이크, 립 전문점입니다.\n\n스터번\n\n* 도움되는 찐리뷰\n\n친구들이랑 방문했고, 분위기도 적당하고 맛도 좋았습니다. 예약도 친절히 잘 받아주시고 파스타도 맛이 좋았고, 소스가 맛있어서 스테이크 맛이 더 살아있었습니다.\n\n\n\n즐겁고 행복한 맛있는 저녁식사 시간이였습니다.\n\n\n\n\n\n스테이크 맛집 찾다가 예약하고 가게 되었는데 정말 스테이크 맛집이였습니다. 그외 다른 음식들도 맛있었습니다. 일하시는 분들도 너무 친절하시고 평일 점심시간에도 사람이 많습니다. 역시 역삼역 맛집이네요.\n\n* 특징 [...] 아보카도 딥은 건강한 맛이었고, 치킨 시저 샐러드가 맛있었습니다. 역시 역삼역 맛집! 링귀니 파스타는 약간 새콤짭짤 특이한 맛입니다. 담에 가면 팬케이크를 먹어봐야겠습니다.\n\n* 특징\n\n주소는 서울 강남구 역삼동 736-1, 운영시간은 매일 9시~22시, 라스트오더는 20시50분입니다. 업체번호는 0507-1312-9012, 메뉴와 가격은 빌즈 리코타 핫케이크 19,800원/ 풀 오지 25,000원등 여러메뉴가 있습니다. 단체석, 주차, 포장, 무선 인터넷이 가능합니다.\n\n\n\n3. 역삼역 맛집\xa0" 알라보 강남점\xa0 "\n\n-\xa0세계인이 사랑하는 슈퍼푸드 아보카도를 모티